In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import csv
import datetime as dt
import random
import numpy as np

In [3]:
#Contains play by play data from 2015-2023 NFL seasons

df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

In [4]:
"""
    Creates variables xp_attempt_perc, xp2_attempt_perc, xp_make_perc, xp2_make_perc
    Stored in arrays based on score difference
    
    i.e. xp_attempt_perc[-8] returns the real percentage that teams down 8 attempt an extra 
        point from the 2015-2023 NFL seasons. Similarly, xp_make_perc[-8] shows the percentage
        of successful extra points when a team is down 8 in the 2015-2023 seasons. xp2_attempt_perc
        and xp2_make_perc work the same way storing information on 2 point conversions. 
    
    Parameters:
    - None

    Returns:
    - None
"""

#Initializes each value to 0. Score difference spans from [-70 to 70].
ScoreDiff = [x for x in range(-70, 71)]
XP_Attempt_Count={}
XP2_Attempt_Count={}
for sd in ScoreDiff:
    XP_Attempt_Count[sd] = 0
    XP2_Attempt_Count[sd] = 0

xp_attempt_perc = {}
xp2_attempt_perc = {}

xp_makes=0
xp2_makes=0

for df in df_list:

    df = pd.read_csv(df)

    for index, row in df.iterrows():

        #Counts for each attempt and make for both extra points and two point conversions
        
        if(row['extra_point_attempt']==1):
            XP_Attempt_Count[int(row['score_differential_post'])]+=1
        if(row['two_point_attempt']==1):
            XP2_Attempt_Count[int(row['score_differential_post'])]+=1
        if(row['extra_point_result']=='good'):
            xp_makes+=1
        if(row['two_point_conv_result']=='success'):
            xp2_makes+=1


    #Calculate attempt percentages at each score difference for both extra points and two point conversions
    for sd in XP_Attempt_Count:
        try:
            xp_attempt_perc[sd] = XP_Attempt_Count[sd]/(XP_Attempt_Count[sd]+XP2_Attempt_Count[sd])
            xp2_attempt_perc[sd] = XP2_Attempt_Count[sd]/(XP_Attempt_Count[sd]+XP2_Attempt_Count[sd])

        except ZeroDivisionError:
            XP_Attempt_Count[sd]=0
            XP2_Attempt_Count[sd]=0

#Calcualte overall make percentage
xp_make_perc = xp_makes/sum(XP_Attempt_Count.values())
xp2_make_perc = xp2_makes/sum(XP2_Attempt_Count.values())

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\260490637.py:33: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,219,220,221,234,235,236,237,238,239,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)
C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\260490637.py:33: DtypeWarning: Columns (180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,219,220,221,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)
C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\260490637.py:33: DtypeWarning: Columns (180,181,183,184,190,191,192,193,194,195,198,199,200,201,204,205,206,207,214,215,219,220,221,249,250,254,255,256,261,263,264,267,268,269,270,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)
C:\Users\lcm021\AppData\L

In [5]:
ot = pd.read_csv("All NFL Game Data (Including OT Stats).csv")

#Creates dictionary with keys of all overtime final scores
#Note 2023 rules allow for possible other variations, but it has not occurred

keylist = ['0-0', '2-0', '3-0', '6-0', '3-3', '6-3']
ot_count=0
ot_scores = {}
for i in keylist:
    ot_scores[i] = 0

#Counts the total of each key
for index, row in ot.iterrows():
    if (int(row['SEASON'])>=2015 and row['OVERTIME']):
        ot_count+=1
        ot_scores[row['OT SCORE']]+=1

#Divides by 2 as the csv has the final score for both teams 
for score in ot_scores:
    ot_scores[score]=ot_scores[score]/2



C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\318916883.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ot = pd.read_csv("All NFL Game Data (Including OT Stats).csv")


In [6]:
print(ot_scores)

{'0-0': 7.0, '2-0': 0.0, '3-0': 74.0, '6-0': 49.0, '3-3': 2.0, '6-3': 13.0}


In [7]:
def ot_scoring(t1, t2, avg_mu):

    """
    Simulates Overtime

    Selects the scoring team based off weights from all scoring events. Then selects the final score based off percentages from the
    2015-2023 seasons. 

    Parameters:
    - t1 (int): Rank of the first team
    - t2 (int): Rank of the second team
    - avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]

    Returns:
    - (tuple): Returns the selected scoring team and the final score after overtime
    """

    w1=0
    w2=0
    for period in range(6):
        w1+=(avg_mu[t1][period][0]+avg_mu[t1][period][1]+avg_mu[t1][period][2]+avg_mu[t1][period][3]+avg_mu[t1][period][4]+avg_mu[t1][period][5])
        w2+=(avg_mu[t2][period][0]+avg_mu[t2][period][1]+avg_mu[t2][period][2]+avg_mu[t2][period][3]+avg_mu[t2][period][4]+avg_mu[t2][period][5])

    weight1=w1/(w1+w2)
    weight2=1-weight1

    scoring_team = random.choices([1,2], weights=(weight1, weight2))
    scoring_team = scoring_team[0]


    ot_event = random.choices([1,2,3,4,5,6], weights=(ot_scores['0-0']/ot_count, ot_scores['2-0']/ot_count, ot_scores['3-0']/ot_count, ot_scores['6-0']/ot_count, ot_scores['3-3']/ot_count, ot_scores['6-3']/ot_count))
    ot_event = ot_event[0]


    if ot_event == 1:
        return (scoring_team, [0,0])
    if ot_event == 2:
        return (scoring_team, [2,0])
    if ot_event == 3:
        return (scoring_team, [3,0])
    if ot_event == 4:
        return (scoring_team, [6,0])
    if ot_event == 5:
        return (scoring_team, [3,3])
    if ot_event == 6:
        return (scoring_team, [6,3])

In [9]:
"""
Stores information on team scoring events and period for every season

i.e.'LAC': [11, 11, 11, 9, 15, 4] means the Chargers had 11 scoring events in the first qtr, 11 scoring events in the second qtr - 2 minute warning, etc.
"""

team_list = ['LAC', 'TEN', 'JAX', 'TB', 'HOU', 'LA', 'LV', 'NE', 'BUF', 'CIN', 'BAL', 'PIT', 'ARI', 'ATL', 'NO', 'IND', 'KC', 'CAR', 'MIA', 'PHI', 'DAL', 'SF', 'SEA', 'WAS', 'DEN', 'NYJ', 'CLE', 'MIN', 'DET', 'CHI', 'NYG', 'GB']
df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

period_dict_list=[]

year_count=2015

for df in df_list:

    df = pd.read_csv(df)

    team_period_dict={}

    for team in team_list:
        team_period_dict[team]=[0,0,0,0,0,0]

    for index, row in df.iterrows():

        team=''

        if row['season_type']=='REG':

            if ((row['touchdown'] == 1 or row['field_goal_result']=='made' or row['safety']==1 or row['defensive_two_point_conv']==1)):

                if row['posteam_score_post']>row['posteam_score']:
                    team='posteam'
                elif row['defteam_score_post']>row['defteam_score']:
                    team='defteam'


                if(row['qtr']==1):
                    team_period_dict[row[team]][0]+=1

                if(row['qtr']==2):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][1]+=1
                    else:
                        team_period_dict[row[team]][2]+=1

                if(row['qtr']==3):
                    team_period_dict[row[team]][3]+=1


                if(row['qtr']==4):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][4]+=1
                    else:
                        team_period_dict[row[team]][5]+=1


    print(year_count, team_period_dict)
    period_dict_list.append(team_period_dict)
    year_count+=1

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\3844262644.py:16: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,219,220,221,234,235,236,237,238,239,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2015 {'LAC': [11, 11, 11, 9, 15, 4], 'TEN': [12, 10, 3, 14, 9, 2], 'JAX': [9, 16, 8, 17, 14, 6], 'TB': [14, 16, 5, 14, 10, 6], 'HOU': [11, 12, 4, 14, 19, 2], 'LA': [13, 14, 5, 11, 7, 2], 'LV': [10, 19, 5, 17, 9, 4], 'NE': [15, 22, 7, 20, 17, 5], 'BUF': [15, 13, 8, 14, 17, 2], 'CIN': [14, 17, 5, 16, 16, 4], 'BAL': [12, 12, 8, 10, 17, 6], 'PIT': [14, 15, 10, 15, 23, 3], 'ARI': [14, 21, 8, 24, 18, 3], 'ATL': [9, 16, 8, 9, 12, 8], 'NO': [19, 12, 5, 15, 13, 5], 'IND': [13, 18, 1, 10, 16, 3], 'KC': [16, 19, 9, 12, 19, 1], 'CAR': [17, 21, 7, 22, 16, 4], 'MIA': [9, 11, 7, 10, 13, 2], 'PHI': [11, 15, 5, 16, 17, 2], 'DAL': [13, 10, 6, 10, 13, 5], 'SF': [6, 12, 7, 10, 8, 3], 'SEA': [14, 13, 10, 20, 18, 3], 'WAS': [17, 14, 8, 11, 15, 6], 'DEN': [13, 13, 7, 11, 17, 5], 'NYJ': [15, 11, 7, 15, 17, 4], 'CLE': [10, 13, 8, 8, 14, 2], 'MIN': [15, 11, 8, 16, 19, 2], 'DET': [15, 12, 8, 10, 15, 5], 'CHI': [10, 15, 8, 11, 17, 6], 'NYG': [15, 17, 9, 14, 17, 6], 'GB': [18, 8, 4, 15, 17, 4]}


C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\3844262644.py:16: DtypeWarning: Columns (180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,219,220,221,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2016 {'LAC': [13, 19, 6, 20, 15, 1], 'TEN': [14, 14, 9, 11, 16, 4], 'JAX': [9, 14, 6, 13, 16, 3], 'TB': [13, 13, 6, 17, 13, 2], 'HOU': [9, 11, 6, 14, 18, 2], 'LA': [13, 6, 5, 6, 11, 2], 'LV': [13, 15, 10, 16, 18, 3], 'NE': [23, 15, 5, 19, 18, 1], 'BUF': [14, 13, 6, 17, 12, 6], 'CIN': [17, 13, 7, 13, 12, 2], 'BAL': [18, 8, 14, 11, 18, 2], 'PIT': [16, 10, 8, 13, 18, 6], 'ARI': [9, 14, 13, 13, 21, 3], 'ATL': [25, 23, 7, 22, 17, 3], 'NO': [15, 19, 9, 14, 24, 3], 'IND': [16, 18, 6, 12, 18, 4], 'KC': [15, 15, 6, 11, 18, 7], 'CAR': [15, 17, 4, 17, 15, 3], 'MIA': [7, 15, 3, 18, 12, 5], 'PHI': [12, 14, 9, 16, 19, 3], 'DAL': [15, 19, 7, 14, 18, 2], 'SF': [13, 16, 3, 6, 14, 3], 'SEA': [13, 19, 8, 10, 18, 2], 'WAS': [14, 12, 7, 19, 21, 4], 'DEN': [8, 14, 7, 7, 22, 6], 'NYJ': [10, 15, 4, 8, 16, 1], 'CLE': [12, 11, 8, 8, 8, 2], 'MIN': [11, 15, 5, 13, 16, 4], 'DET': [11, 13, 9, 11, 14, 10], 'CHI': [9, 13, 6, 8, 13, 2], 'NYG': [9, 13, 6, 12, 13, 4], 'GB': [15, 17, 9, 12, 23, 1]}


C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\3844262644.py:16: DtypeWarning: Columns (180,181,183,184,190,191,192,193,194,195,198,199,200,201,204,205,206,207,214,215,219,220,221,249,250,254,255,256,261,263,264,267,268,269,270,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


...

2021 {'LAC': [14, 19, 7, 12, 22, 7], 'TEN': [11, 23, 8, 10, 17, 4], 'JAX': [10, 9, 7, 8, 12, 3], 'TB': [16, 20, 8, 18, 20, 5], 'HOU': [10, 15, 7, 6, 14, 1], 'LA': [13, 18, 11, 18, 17, 7], 'LV': [13, 11, 11, 16, 16, 5], 'NE': [12, 18, 10, 17, 28, 2], 'BUF': [18, 15, 10, 18, 21, 3], 'CIN': [13, 12, 13, 20, 19, 4], 'BAL': [9, 15, 10, 15, 15, 10], 'PIT': [7, 11, 4, 12, 30, 4], 'ARI': [17, 15, 9, 23, 15, 3], 'ATL': [13, 15, 10, 10, 8, 5], 'NO': [8, 12, 9, 9, 24, 6], 'IND': [17, 15, 11, 18, 20, 1], 'KC': [21, 15, 8, 17, 21, 2], 'CAR': [17, 15, 5, 6, 15, 1], 'MIA': [15, 11, 5, 8, 19, 3], 'PHI': [15, 14, 7, 20, 21, 3], 'DAL': [21, 15, 12, 18, 18, 8], 'SF': [15, 11, 12, 18, 15, 4], 'SEA': [14, 13, 11, 13, 14, 3], 'WAS': [14, 13, 8, 11, 14, 5], 'DEN': [9, 15, 7, 12, 18, 2], 'NYJ': [10, 10, 6, 11, 16, 3], 'CLE': [12, 12, 10, 12, 12, 1], 'MIN': [16, 17, 6, 20, 15, 6], 'DET': [8, 13, 8, 13, 13, 6], 'CHI': [9, 12, 11, 11, 10, 7], 'NYG': [8, 15, 0, 9, 18, 3], 'GB': [9, 19, 11, 20, 18, 1]}


C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\3844262644.py:16: DtypeWarning: Columns (46,180,181,183,184,188,189,190,191,194,195,198,199,204,205,206,207,214,215,219,220,221,223,225,227,244,245,246,249,250,254,255,256,261,263,264,267,268,269,270,284,285) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2022 {'LAC': [10, 24, 11, 11, 12, 5], 'TEN': [13, 16, 7, 10, 7, 1], 'JAX': [12, 18, 7, 18, 16, 3], 'TB': [13, 10, 10, 8, 16, 5], 'HOU': [10, 16, 2, 17, 11, 3], 'LA': [16, 13, 7, 11, 10, 3], 'LV': [15, 16, 10, 13, 16, 5], 'NE': [10, 15, 7, 22, 15, 2], 'BUF': [16, 16, 10, 20, 13, 7], 'CIN': [16, 16, 8, 16, 16, 2], 'BAL': [14, 17, 6, 16, 16, 2], 'PIT': [12, 14, 10, 10, 11, 5], 'ARI': [10, 15, 10, 10, 17, 2], 'ATL': [13, 11, 10, 15, 17, 4], 'NO': [11, 13, 5, 14, 14, 3], 'IND': [11, 12, 6, 16, 11, 3], 'KC': [17, 19, 10, 18, 18, 2], 'CAR': [9, 16, 5, 11, 21, 7], 'MIA': [15, 22, 6, 15, 9, 6], 'PHI': [14, 25, 11, 14, 15, 2], 'DAL': [16, 15, 10, 18, 22, 3], 'SF': [15, 16, 9, 18, 19, 2], 'SEA': [19, 14, 8, 15, 19, 2], 'WAS': [10, 10, 8, 12, 18, 4], 'DEN': [12, 13, 9, 8, 15, 1], 'NYJ': [12, 10, 9, 9, 14, 7], 'CLE': [12, 14, 5, 14, 16, 4], 'MIN': [13, 14, 7, 10, 24, 6], 'DET': [19, 16, 8, 13, 19, 4], 'CHI': [17, 11, 7, 16, 10, 1], 'NYG': [8, 13, 7, 18, 17, 6], 'GB': [15, 14, 7, 13, 15, 2]}


C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\3844262644.py:16: DtypeWarning: Columns (180,181,183,184,204,205,206,207,219,220,221,223,225,227,254,255,256,261,263,264,284,285,333) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2023 {'LAC': [13, 12, 6, 7, 12, 1], 'TEN': [11, 9, 5, 10, 7, 1], 'JAX': [10, 13, 5, 13, 15, 2], 'TB': [7, 10, 4, 8, 13, 2], 'HOU': [10, 13, 6, 11, 13, 3], 'LA': [12, 8, 10, 11, 12, 5], 'LV': [10, 9, 5, 6, 9, 2], 'NE': [6, 6, 2, 6, 8, 1], 'BUF': [11, 14, 6, 11, 15, 3], 'CIN': [11, 8, 4, 12, 14, 2], 'BAL': [15, 12, 8, 14, 14, 1], 'PIT': [5, 10, 2, 11, 11, 2], 'ARI': [12, 10, 8, 8, 4, 3], 'ATL': [7, 10, 3, 10, 13, 4], 'NO': [11, 14, 2, 13, 13, 1], 'IND': [11, 13, 7, 10, 13, 2], 'KC': [11, 12, 14, 9, 9, 0], 'CAR': [6, 11, 3, 9, 8, 3], 'MIA': [13, 12, 10, 15, 11, 1], 'PHI': [15, 7, 5, 14, 14, 3], 'DAL': [17, 16, 8, 10, 19, 1], 'SF': [12, 14, 7, 11, 16, 0], 'SEA': [13, 9, 6, 12, 9, 2], 'WAS': [8, 9, 7, 10, 9, 4], 'DEN': [12, 8, 9, 6, 16, 6], 'NYJ': [6, 11, 5, 9, 9, 5], 'CLE': [9, 13, 9, 13, 11, 4], 'MIN': [6, 14, 6, 13, 7, 2], 'DET': [15, 13, 6, 7, 12, 5], 'CHI': [10, 13, 4, 10, 16, 1], 'NYG': [4, 8, 2, 8, 9, 4], 'GB': [9, 7, 4, 17, 13, 2]}


In [10]:
"""
Stores information on total scoring events per team in a season

i.e. {2015: [(('LAC'), 61)] means the Chargers had 61 total scoring events in 2015
"""

team_list = period_dict_list[0].keys()

team_total_scoring_events = {
    2015: [],
    2016: [],
    2017: [],
    2018: [],
    2019: [],
    2020: [],
    2021: [],
    2022: [],
    2023: []
}

season_count=0
for key in team_total_scoring_events:

    for team in team_list:

        team_total_scoring_events[key]+=[(team, sum(period_dict_list[season_count][team]))]

    season_count+=1

print(team_total_scoring_events)

{2015: [('LAC', 61), ('TEN', 50), ('JAX', 70), ('TB', 65), ('HOU', 62), ('LA', 52), ('LV', 64), ('NE', 86), ('BUF', 69), ('CIN', 72), ('BAL', 65), ('PIT', 80), ('ARI', 88), ('ATL', 62), ('NO', 69), ('IND', 61), ('KC', 76), ('CAR', 87), ('MIA', 52), ('PHI', 66), ('DAL', 57), ('SF', 46), ('SEA', 78), ('WAS', 71), ('DEN', 66), ('NYJ', 69), ('CLE', 55), ('MIN', 71), ('DET', 65), ('CHI', 67), ('NYG', 78), ('GB', 66)], 2016: [('LAC', 74), ('TEN', 68), ('JAX', 61), ('TB', 64), ('HOU', 60), ('LA', 43), ('LV', 75), ('NE', 81), ('BUF', 68), ('CIN', 64), ('BAL', 71), ('PIT', 71), ('ARI', 73), ('ATL', 97), ('NO', 84), ('IND', 74), ('KC', 72), ('CAR', 71), ('MIA', 60), ('PHI', 73), ('DAL', 75), ('SF', 55), ('SEA', 70), ('WAS', 77), ('DEN', 64), ('NYJ', 54), ('CLE', 49), ('MIN', 64), ('DET', 68), ('CHI', 51), ('NYG', 57), ('GB', 77)], 2017: [('LAC', 63), ('TEN', 67), ('JAX', 77), ('TB', 63), ('HOU', 61), ('LA', 93), ('LV', 52), ('NE', 88), ('BUF', 59), ('CIN', 52), ('BAL', 76), ('PIT', 79), ('ARI', 

In [11]:
"""
Stores information on scoring events per team per period in a season

i.e. 'LAC': [[1, 4, 0, 6, 0, 0], [1, 5, 0, 5, 0, 0], [1, 5, 0, 5, 0, 0], [0, 6, 0, 3, 0, 0], [2, 7, 1, 5, 0, 0], [0, 1, 1, 2, 0, 0]]
means that the Chargers had 1 TD6, 4 TD7, 0 TD8, 6 FG, 0 SFTY, and 0 D2P in the first quarter, etc.
"""

team_list = ['LAC', 'TEN', 'JAX', 'TB', 'HOU', 'LA', 'LV', 'NE', 'BUF', 'CIN', 'BAL', 'PIT', 'ARI', 'ATL', 'NO', 'IND', 'KC', 'CAR', 'MIA', 'PHI', 'DAL', 'SF', 'SEA', 'WAS', 'DEN', 'NYJ', 'CLE', 'MIN', 'DET', 'CHI', 'NYG', 'GB']
df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

period_dict_list=[]

year_count=2015
event_index=0
scoring_event=False
team=''

for df in df_list:

    df = pd.read_csv(df)

    team_period_dict={}

    for team in team_list:
        team_period_dict[team]=[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    for index, row in df.iterrows():

        scoring_event=False

        if row['season_type']=='REG':

            #TD 6
            if ((row['extra_point_result'] == 'failed' or row['extra_point_result'] == 'blocked' or row['two_point_conv_result']=='failure')):
                event_index=0
                scoring_event=True

            #TD 7
            if (row['extra_point_result'] == 'good'):
                event_index=1
                scoring_event=True

            #TD8
            if(row['two_point_conv_result']=='success'):
                event_index=2
                scoring_event=True

            #FG
            if (row['field_goal_result']=='made'):
                event_index=3
                scoring_event=True

            #SFTY
            if (row['safety']==1):
                event_index=4
                scoring_event=True

            #DP2
            if (row['defensive_two_point_conv']==1):
                event_index=5
                scoring_event=True

            if scoring_event==True:


                if row['defteam_score_post']>row['defteam_score']:
                    team='defteam'
                else:
                    team='posteam'

                if(row['qtr']==1):
                    team_period_dict[row[team]][0][event_index]+=1

                if(row['qtr']==2):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][1][event_index]+=1
                    else:
                        team_period_dict[row[team]][2][event_index]+=1

                if(row['qtr']==3):
                    team_period_dict[row[team]][3][event_index]+=1

                if(row['qtr']==4):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][4][event_index]+=1
                    else:
                        team_period_dict[row[team]][5][event_index]+=1


    print(year_count, team_period_dict)
    period_dict_list.append(team_period_dict)
    year_count+=1

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\1914210160.py:20: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,219,220,221,234,235,236,237,238,239,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2015 {'LAC': [[1, 4, 0, 6, 0, 0], [1, 5, 0, 5, 0, 0], [1, 5, 0, 5, 0, 0], [0, 6, 0, 3, 0, 0], [2, 7, 1, 5, 0, 0], [0, 1, 1, 2, 0, 0]], 'TEN': [[1, 7, 0, 4, 0, 0], [0, 5, 0, 5, 0, 0], [0, 3, 0, 0, 0, 0], [0, 8, 2, 4, 0, 0], [1, 6, 1, 1, 0, 0], [2, 0, 0, 0, 0, 0]], 'JAX': [[0, 2, 0, 7, 0, 0], [2, 7, 0, 6, 0, 0], [4, 2, 0, 3, 0, 0], [1, 10, 0, 6, 0, 0], [3, 8, 0, 2, 0, 0], [1, 3, 1, 2, 0, 0]], 'TB': [[0, 4, 0, 10, 0, 0], [1, 9, 0, 5, 0, 0], [1, 3, 0, 2, 0, 0], [1, 6, 1, 6, 0, 0], [1, 5, 0, 4, 0, 0], [1, 4, 0, 1, 0, 0]], 'HOU': [[2, 5, 0, 4, 0, 0], [0, 6, 0, 6, 0, 0], [0, 1, 0, 3, 0, 0], [1, 9, 0, 4, 0, 0], [2, 11, 2, 4, 0, 0], [0, 0, 0, 2, 0, 0]], 'LA': [[1, 7, 0, 4, 1, 0], [1, 5, 1, 7, 0, 0], [0, 2, 0, 3, 0, 0], [1, 7, 0, 3, 0, 0], [0, 5, 0, 2, 0, 0], [0, 1, 0, 1, 0, 0]], 'LV': [[0, 7, 0, 3, 0, 0], [0, 11, 0, 7, 0, 0], [0, 5, 0, 1, 0, 0], [1, 8, 0, 6, 2, 0], [2, 4, 1, 2, 0, 0], [0, 3, 0, 1, 0, 0]], 'NE': [[0, 11, 0, 4, 0, 0], [0, 11, 0, 10, 1, 0], [0, 4, 0, 3, 0, 0], [0, 10, 0, 10, 0, 0]

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\1914210160.py:20: DtypeWarning: Columns (180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,219,220,221,249,250,254,255,256,261,263,264,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2016 {'LAC': [[1, 9, 0, 3, 0, 0], [0, 11, 0, 7, 0, 0], [0, 5, 0, 2, 0, 0], [4, 10, 0, 6, 0, 0], [1, 7, 0, 6, 1, 0], [0, 0, 0, 1, 0, 0]], 'TEN': [[1, 8, 0, 5, 0, 0], [0, 12, 0, 1, 0, 0], [0, 8, 0, 2, 0, 0], [2, 5, 0, 4, 0, 0], [2, 6, 0, 8, 0, 0], [2, 0, 0, 2, 0, 0]], 'JAX': [[0, 6, 0, 3, 0, 0], [1, 3, 0, 8, 0, 0], [1, 3, 0, 4, 0, 0], [0, 5, 1, 7, 0, 0], [1, 10, 1, 4, 0, 0], [0, 2, 0, 1, 0, 0]], 'TB': [[1, 6, 0, 6, 0, 0], [0, 9, 0, 4, 0, 0], [0, 2, 0, 4, 0, 0], [0, 11, 1, 4, 1, 0], [4, 4, 2, 3, 0, 0], [0, 0, 1, 1, 0, 0]], 'HOU': [[0, 5, 0, 4, 0, 0], [0, 6, 0, 5, 0, 0], [0, 0, 0, 5, 1, 0], [1, 5, 0, 8, 0, 0], [1, 5, 0, 11, 0, 0], [1, 1, 0, 1, 0, 0]], 'LA': [[0, 8, 0, 5, 0, 0], [0, 2, 0, 3, 0, 0], [0, 3, 0, 3, 0, 0], [0, 2, 0, 4, 0, 0], [1, 6, 0, 4, 0, 0], [0, 2, 0, 0, 0, 0]], 'LV': [[0, 5, 0, 8, 0, 0], [1, 7, 0, 7, 0, 0], [1, 6, 0, 3, 0, 0], [1, 8, 1, 6, 0, 0], [1, 10, 3, 4, 0, 0], [0, 1, 1, 1, 0, 0]], 'NE': [[1, 15, 0, 5, 2, 0], [0, 10, 0, 5, 0, 0], [0, 3, 0, 2, 0, 0], [2, 11, 0, 5, 1, 0

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\1914210160.py:20: DtypeWarning: Columns (180,181,183,184,190,191,192,193,194,195,198,199,200,201,204,205,206,207,214,215,219,220,221,249,250,254,255,256,261,263,264,267,268,269,270,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


...

2021 {'LAC': [[1, 9, 0, 4, 0, 0], [4, 7, 3, 4, 0, 0], [0, 6, 0, 2, 0, 0], [1, 2, 1, 8, 0, 0], [2, 14, 3, 3, 0, 0], [3, 2, 0, 2, 0, 0]], 'TEN': [[0, 7, 0, 4, 0, 0], [2, 13, 0, 8, 0, 0], [0, 4, 0, 4, 0, 0], [1, 8, 0, 1, 0, 0], [0, 7, 2, 6, 0, 0], [1, 4, 0, 1, 0, 0]], 'JAX': [[2, 3, 0, 5, 0, 0], [1, 5, 0, 3, 0, 0], [1, 2, 0, 4, 0, 0], [1, 3, 1, 3, 0, 0], [3, 3, 1, 5, 0, 0], [0, 2, 0, 1, 0, 0]], 'TB': [[1, 14, 0, 1, 0, 0], [0, 10, 0, 8, 0, 0], [1, 7, 0, 2, 0, 0], [0, 15, 0, 3, 0, 0], [3, 8, 0, 9, 0, 0], [0, 2, 1, 2, 0, 0]], 'HOU': [[1, 7, 0, 1, 1, 0], [2, 3, 0, 9, 0, 0], [1, 3, 0, 4, 0, 0], [0, 4, 0, 2, 0, 0], [0, 7, 2, 5, 0, 0], [0, 1, 0, 0, 0, 0]], 'LA': [[0, 6, 0, 7, 0, 0], [0, 11, 0, 7, 0, 0], [0, 6, 0, 5, 0, 0], [1, 14, 0, 3, 0, 0], [0, 9, 2, 6, 0, 0], [1, 2, 0, 4, 0, 0]], 'LV': [[0, 7, 0, 5, 1, 0], [0, 4, 0, 6, 0, 0], [0, 4, 0, 8, 0, 0], [2, 10, 0, 4, 0, 0], [4, 5, 0, 7, 0, 0], [0, 0, 0, 5, 0, 0]], 'NE': [[0, 9, 1, 2, 0, 0], [1, 11, 0, 6, 0, 0], [0, 3, 0, 7, 0, 0], [2, 5, 0, 10, 0, 0

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\1914210160.py:20: DtypeWarning: Columns (46,180,181,183,184,188,189,190,191,194,195,198,199,204,205,206,207,214,215,219,220,221,223,225,227,244,245,246,249,250,254,255,256,261,263,264,267,268,269,270,284,285) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2022 {'LAC': [[0, 5, 0, 4, 0, 0], [0, 19, 0, 5, 0, 0], [0, 3, 0, 8, 0, 0], [0, 4, 0, 7, 0, 0], [0, 6, 1, 4, 1, 0], [0, 2, 1, 2, 0, 0]], 'TEN': [[0, 11, 0, 2, 0, 0], [0, 10, 0, 6, 0, 0], [0, 3, 0, 4, 0, 0], [1, 6, 0, 3, 0, 0], [0, 1, 1, 5, 0, 0], [0, 1, 0, 0, 0, 0]], 'JAX': [[0, 6, 1, 4, 0, 0], [0, 9, 0, 8, 0, 0], [0, 2, 0, 5, 0, 0], [3, 8, 2, 5, 0, 0], [1, 9, 0, 6, 0, 0], [0, 0, 1, 2, 0, 0]], 'TB': [[0, 3, 0, 10, 0, 0], [0, 6, 0, 4, 0, 0], [0, 3, 0, 7, 0, 0], [0, 4, 1, 3, 0, 0], [2, 5, 1, 6, 0, 0], [4, 2, 0, 0, 0, 0]], 'HOU': [[0, 7, 0, 4, 0, 0], [0, 8, 0, 8, 1, 0], [0, 1, 0, 1, 0, 0], [1, 7, 0, 10, 0, 0], [2, 4, 0, 5, 0, 0], [1, 1, 1, 1, 0, 0]], 'LA': [[1, 7, 0, 9, 0, 0], [0, 8, 0, 5, 0, 0], [0, 5, 0, 3, 0, 0], [1, 4, 0, 6, 0, 0], [0, 6, 1, 4, 0, 0], [0, 2, 0, 1, 0, 0]], 'LV': [[0, 8, 0, 8, 0, 0], [1, 10, 0, 5, 0, 0], [0, 4, 0, 6, 0, 0], [0, 7, 0, 7, 0, 0], [3, 5, 0, 7, 0, 0], [1, 3, 0, 1, 0, 0]], 'NE': [[0, 4, 0, 6, 0, 0], [0, 8, 0, 7, 0, 0], [0, 1, 0, 5, 0, 0], [2, 9, 0, 9, 0, 0], [

C:\Users\lcm021\AppData\Local\Temp\ipykernel_9112\1914210160.py:20: DtypeWarning: Columns (180,181,183,184,204,205,206,207,219,220,221,223,225,227,254,255,256,261,263,264,284,285,333) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df)


2023 {'LAC': [[0, 8, 0, 5, 0, 0], [0, 7, 1, 4, 0, 0], [0, 3, 0, 3, 0, 0], [0, 3, 0, 4, 0, 0], [0, 10, 0, 2, 0, 0], [0, 0, 0, 1, 0, 0]], 'TEN': [[0, 3, 0, 8, 0, 0], [0, 7, 0, 2, 0, 0], [0, 1, 0, 4, 0, 0], [0, 5, 0, 4, 0, 1], [1, 3, 0, 3, 0, 0], [0, 0, 0, 1, 0, 0]], 'JAX': [[0, 5, 1, 4, 0, 0], [0, 8, 0, 5, 0, 0], [0, 1, 0, 4, 0, 0], [0, 8, 2, 3, 0, 0], [0, 8, 0, 7, 0, 0], [1, 0, 0, 1, 0, 0]], 'TB': [[0, 3, 0, 4, 0, 0], [0, 6, 0, 4, 0, 0], [0, 3, 0, 1, 0, 0], [0, 4, 0, 4, 0, 0], [1, 5, 2, 5, 0, 0], [0, 1, 0, 1, 0, 0]], 'HOU': [[0, 7, 0, 3, 0, 0], [0, 6, 0, 7, 0, 0], [0, 1, 0, 5, 0, 0], [4, 3, 0, 4, 0, 0], [1, 6, 1, 5, 0, 0], [1, 1, 0, 1, 0, 0]], 'LA': [[0, 6, 0, 6, 0, 0], [0, 4, 0, 4, 0, 0], [2, 4, 0, 4, 0, 0], [0, 3, 2, 5, 1, 0], [1, 5, 0, 4, 0, 0], [0, 2, 0, 3, 1, 0]], 'LV': [[0, 7, 0, 3, 0, 0], [0, 5, 0, 3, 0, 0], [0, 1, 0, 5, 0, 0], [0, 1, 0, 5, 0, 0], [0, 4, 1, 4, 0, 0], [1, 0, 0, 0, 1, 0]], 'NE': [[0, 2, 0, 4, 0, 0], [0, 4, 0, 2, 0, 0], [0, 1, 0, 1, 0, 0], [0, 2, 0, 4, 0, 0], [2, 4,

In [15]:
'''
The overall scoring event rate in period p of scoring event type e for the r-th best team

2015 mu[year(2015-2023)][ranking(1-32)][period(1-6][event(1-6)]

mu[2015][0][0][0] show best TD6 scoring rate in first quarter in 2015
'''

mu = [[[[0 for _ in range(6)] for _ in range(6)] for _ in range(32)] for _ in range(9)]

for year in range(9):
        for period in range(6):
            for event in range(6):
                sorted_list = sorted(period_dict_list[year].items(), key=lambda x: x[1][period][event], reverse=True)
                for rank in range(32):
                    mu[year][rank][period][event]=sorted_list[rank][1][period][event]


In [20]:
# Overall Mu (Combining All Seasons)

avg_mu = [[[0 for _ in range(6)] for _ in range(6)] for _ in range(32)]
games=0


for period in range(6):
    for event in range(6):
        for rank in range(32):
            for year in range(9):

                avg_mu[rank][period][event] += mu[year][rank][period][event]

            avg_mu[rank][period][event] = avg_mu[rank][period][event]/147


In [22]:
# Get Average Scoring Rate Per Period

period1_total = 0
period2_total = 0
period3_total = 0
period4_total = 0
period5_total = 0
period6_total = 0

for rank in range(32):
    period1_total+=avg_mu[rank][0][0]+avg_mu[rank][0][1]+avg_mu[rank][0][2]+avg_mu[rank][0][3]+avg_mu[rank][0][4]+avg_mu[rank][0][5]
    period2_total+=avg_mu[rank][1][0]+avg_mu[rank][1][1]+avg_mu[rank][1][2]+avg_mu[rank][1][3]+avg_mu[rank][1][4]+avg_mu[rank][1][5]
    period3_total+=avg_mu[rank][2][0]+avg_mu[rank][2][1]+avg_mu[rank][2][2]+avg_mu[rank][2][3]+avg_mu[rank][2][4]+avg_mu[rank][2][5]
    period4_total+=avg_mu[rank][3][0]+avg_mu[rank][3][1]+avg_mu[rank][3][2]+avg_mu[rank][3][3]+avg_mu[rank][3][4]+avg_mu[rank][3][5]
    period5_total+=avg_mu[rank][4][0]+avg_mu[rank][4][1]+avg_mu[rank][4][2]+avg_mu[rank][4][3]+avg_mu[rank][4][4]+avg_mu[rank][4][5]
    period6_total+=avg_mu[rank][5][0]+avg_mu[rank][5][1]+avg_mu[rank][5][2]+avg_mu[rank][5][3]+avg_mu[rank][5][4]+avg_mu[rank][5][5]

period1_mu = 2 * 1/32*(period1_total)
period2_mu = 2 * 1/32*(period2_total)
period3_mu = 2 * 1/32*(period3_total)
period4_mu = 2 * 1/32*(period4_total)
period5_mu = 2 * 1/32*(period5_total)
period6_mu = 2 * 1/32*(period6_total)

In [23]:
# Scoring Rates Simulation

def sim(avg_mu):

        """
        Simulation

        Simulates an NFL game by generating a specific number of scoring events in each period.
        The period mus are acquired from percentages of real scoring times from 2015-2023.

        Parameters:
        - avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]

        Returns:
        -[s1,s2] (list): Returns a list of one final score in format [team1_score, team2_score]
        """

        stoplist = [15, 28, 30, 45, 58, 60]
        numbers = list(range(0, 32))
        random.shuffle(numbers)

        t1=numbers[0]
        t2=numbers[1]

        t=0
        s1=0
        s2=0
        x=0

        for time in stoplist:

            scoring_events=0

            #First Quarter
            if (time==15):
                x = np.random.poisson(period1_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=0)
                    scoring_events+=1

            #Second Quarter
            elif (time==28):
                x = np.random.poisson(period2_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=1)
                    scoring_events+=1


            #Second Quarter 2 min remaining
            elif (time==30):
                x = np.random.poisson(period3_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=2)
                    scoring_events+=1

            #Third Quarter
            elif (time==45):
                x = np.random.poisson(period4_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=3)
                    scoring_events+=1


            #Fourth Quarter
            elif (time==58):
                x = np.random.poisson(period5_mu)

                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=4)
                    scoring_events+=1


            #Fourth Quarter 2 minutes remaining
            elif (time==60):

                x = np.random.poisson(period6_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=5)
                    scoring_events+=1

        if (time==60 and s1 != s2):
            return([s1,s2])

        else:
            ot_score_updated = ot_scoring(t1, t2, avg_mu)

            #T1 Scores
            if ot_score_updated[0]==1:
                s1 += ot_score_updated[1][0]
                s2 += ot_score_updated[1][1]
            else:
                s1 += ot_score_updated[1][1]
                s2 += ot_score_updated[1][0]
            return([s1,s2])


In [31]:
def score_event_sd(s1, s2, t1, t2, avg_mu, period):


    """
    Score Event Simulation

    Generates which team should score and how much their points increase by

    Parameters:
    - s1 (int): Score of team 1
    - s2 (int): Score of team 2
    - t1 (int): Rank of team 1
    - t2 (int): Rank of team 2
    - avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]
    - period (int): Current game period (1st quarter, 2nd quarter, 2nd quarter 2 min remaining, etc.)

    Returns:
    -[s1,s2] (list): Returns a list of updated score in format [team1_score, team2_score]
    """
    td_rate=0
    fg_rate=0
    sfty_rate=0

    #Time Remaining at start of each period
    time_remaining_list = [60, 45, 32, 30, 13, 2]
    time_remaining = time_remaining_list[period]

    #Hard coded values of delta and lambda
    delta=0.4
    lamb=0.02

    theta = pow(2.718281828459045, ((lamb*-1)*(60-time_remaining)))

    # Calculating Total TD, FG, and SFTY rates over all teams for current period
    for x in range(32):
        td_rate+=avg_mu[x][period][0]+avg_mu[x][period][1]+avg_mu[x][period][2]+avg_mu[x][period][5]
        fg_rate+=avg_mu[x][period][3]
        sfty_rate+=avg_mu[x][period][4]


    #Get avg scoring rate
    td_rate=td_rate/32
    fg_rate=fg_rate/32
    sfty_rate=sfty_rate/32

    #Create mu_bar (average scoring rate of all events in current period)
    mu_bar = 0
    for team in range(32):
        for event in range(6):
            mu_bar+=(avg_mu[team][period][event])
    mu_bar = (mu_bar*1/32)

    #Mu 1 (average scoring rate for all events for team 1 in current period)
    mu_1=0
    for event in range(6):
        mu_1+=avg_mu[t1][period][event]

    #Mu 2 (average scoring rate for all events for team 2 in current period)
    mu_2=0
    for event in range(6):
        mu_2+=avg_mu[t2][period][event]

    w1=0
    w2=0

    #If t1 is beating t2, give advantage to t2
    if s1>s2:
        w1 = mu_1*theta+(1-theta)*mu_bar*(1-delta)
        w2 = mu_2*theta+(1-theta)*mu_bar*(1+delta)

    #If t2 is beating t1, give advantage to t1
    elif s1<s2:
        w1 = mu_1*theta+(1-theta)*mu_bar*(1+delta)
        w2 = mu_2*theta+(1-theta)*mu_bar*(1-delta)

    #If tied, no advantage
    else:
        w1 = mu_1*theta+(1-theta)*mu_bar
        w2 = mu_2*theta+(1-theta)*mu_bar

    weight1=(w1/(w1+w2))
    weight2=1-weight1

    #"Flip Coin" to decide which team scores
    scoring_team = random.choices([1,2], weights=(weight1, weight2))
    scoring_team = scoring_team[0]

    XP_or_2PT=0

    #Choose scoring event based off above rates
    event = random.choices(["FG", "TD", "SFTY"], weights=(fg_rate, td_rate, sfty_rate))
    event=event[0]

   #Field Goal
    if(event=='FG'):
        if (scoring_team==1):
            return[s1+3, s2]
        else:
            return[s1, s2+3]

    #Safety
    elif(event=='SFTY'):
        if (scoring_team==1):
            return[s1+2, s2]
        else:
            return[s1, s2+2]

    #Touchdown
    else:

        if(scoring_team==1):

            #If score difference does not exist kick xp
            if((s1+6)-s2 not in xp_attempt_perc.keys()):
                XP_or_2PT=1

            else:
                #Deciding 1 or 2 point
                XP_or_2PT_Choice = random.choices([1,2], weights=(xp_attempt_perc[(s1+6)-s2], xp2_attempt_perc[(s1+6)-s2]))
                XP_or_2PT=XP_or_2PT_Choice[0]

            #If 1 point, calculate make rate
            if (XP_or_2PT == 1):
                make_xp = random.choices([1,0], weights=(xp_make_perc, 1-xp_make_perc))
                make_xp=make_xp[0]
                return [s1+6+make_xp, s2]

           #If 2 points, calculate make rate
            elif (XP_or_2PT == 2):
                make_xp2 = random.choices([2,0], weights=(xp2_make_perc, 1-xp2_make_perc))
                make_xp2=make_xp2[0]
                return [s1+6+make_xp2, s2]

        #Team 2 Scores
        else:

            #If score difference does not exist kick xp
            if((s2+6)-s1 not in xp_attempt_perc.keys()):
                XP_or_2PT=1

            else:
                #Deciding 1 or 2 point
                XP_or_2PT_Choice = random.choices([1,2], weights=(xp_attempt_perc[(s2+6)-s1], xp2_attempt_perc[(s2+6)-s1]))
                XP_or_2PT=XP_or_2PT_Choice[0]

            #If 1 point, calculate make rate
            if (XP_or_2PT == 1):
                make_xp = random.choices([1,0], weights=(xp_make_perc, 1-xp_make_perc))
                make_xp=make_xp[0]
                return [s1, s2+6+make_xp]


            #If 2 points, calculate make rate
            elif (XP_or_2PT == 2):
                make_xp2 = random.choices([2,0], weights=(xp2_make_perc, 1-xp2_make_perc))
                make_xp2=make_xp2[0]
                return [s1, s2+6+make_xp2]

    return [s1, s2]


In [32]:
"""
Run Many Simulations

Change 

"""


score_count = [ [0 for i in range(j)] for j in range(1, 200)]

for x in range(300):

    game=sim(avg_mu)

    if(x%50==0):
        print(game)

    win_pts=max(game)
    lose_pts=min(game)

    score_count[win_pts][lose_pts]+=1

with open('ScoreDiff_Sim.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    for i in range(len(score_count)):
        writer.writerow(score_count[i])

[35, 41]
[31, 39]
[19, 45]
[17, 9]
[30, 8]
[16, 13]
